# IA: Introducción práctica — **SOLUCIONES**

Versión con celdas resueltas y comentarios para el docente/alumno.
Cumple con: *gráficas con Matplotlib, una figura por celda, sin estilos/colores forzados.*

> **Versión para estudiantes (enunciados):** [IA_intro_colab.ipynb](/mnt/data/IA_intro_colab.ipynb)


## 0) Requisitos
- `numpy`, `matplotlib`, `scikit-learn`, `Pillow`.
- En Colab vienen preinstaladas.

**Opcional (Colab):**
```python
from google.colab import drive
drive.mount('/content/drive')
```

## 1) Cargar y mostrar una imagen (solución)
**Idea clave:** una imagen es una matriz de números.

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Ruta local creada en el entorno de práctica; cambia la ruta si estás en Colab
ruta = '/mnt/data/muestra.png'
try:
    img = Image.open(ruta)
except FileNotFoundError:
    # Crear una imagen simple si no existe (gradient + cuadro)
    import numpy as np
    from PIL import Image, ImageDraw
    arr = np.tile(np.linspace(0, 255, 128, dtype=np.uint8), (128, 1))
    img = Image.fromarray(arr)
    d = ImageDraw.Draw(img)
    d.rectangle([32,32,96,96], outline=255, width=2)
    img.save(ruta)

arr = np.array(img)
print('Forma (alto, ancho):', arr.shape)
plt.imshow(arr)
plt.title('Imagen de ejemplo (sin colormap explícito)')
plt.axis('off')
plt.show()

## 2) Operaciones básicas (solución)
- Brillo = sumar una constante.
- Contraste = escalar alrededor del valor medio.
- Umbral = 0/255 según un límite.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def clip01(x):
    return np.clip(x, 0, 255)

x = arr.astype(float)
brillo = clip01(x + 40)
contraste = clip01((x - 127.5) * 1.3 + 127.5)
umbral = (x > 128).astype(np.uint8) * 255

# Mostrar una figura por celda (regla de estilo)
plt.imshow(brillo)
plt.title('Más brillo')
plt.axis('off')
plt.show()

plt.imshow(contraste)
plt.title('Más contraste')
plt.axis('off')
plt.show()

plt.imshow(umbral)
plt.title('Umbral (B/N)')
plt.axis('off')
plt.show()

## 3) ML básico: clasificación con `digits` (solución)
Flujo: datos → entrenamiento → predicción → métricas.

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Reporte de clasificación:')
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure()
plt.imshow(cm)
plt.title('Matriz de confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.colorbar()
plt.show()

In [ ]:
# Visualizar algunas predicciones
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2,5)
for ax, img, pred, true in zip(axes.ravel(), X_test[:10], y_pred[:10], y_test[:10]):
    ax.imshow(img.reshape(8,8))
    ax.set_title(f'{true}→{pred}')
    ax.axis('off')
plt.tight_layout()
plt.show()

## 4) Red neuronal simple (MLP) — solución

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

mlp = MLPClassifier(hidden_layer_sizes=(64,), activation='relu', max_iter=300, random_state=0)
mlp.fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)
print('Accuracy MLP:', accuracy_score(y_test, pred_mlp))

## 5) Mini-proyecto — solución de referencia
Para facilitar la corrección, generamos un **conjunto sintético** si no se encuentra la carpeta `ok/defecto`.
- Clase `ok`: cuadrados regulares.
- Clase `defecto`: cuadrados con ruido y bordes irregulares.

In [ ]:
from pathlib import Path
from PIL import Image, ImageDraw
import numpy as np

def generar_sintetico(n_por_clase=60, size=(32,32)):
    X, y = [], []
    w, h = size
    for i in range(n_por_clase):
        # ok
        img = Image.new('L', size, 0)
        d = ImageDraw.Draw(img)
        d.rectangle([8,8,w-8,h-8], outline=255, width=2)
        X.append(np.array(img, dtype='float32').reshape(-1))
        y.append(0)
    for i in range(n_por_clase):
        # defecto
        img = Image.new('L', size, 0)
        d = ImageDraw.Draw(img)
        jitter = np.random.randint(-3, 3)
        d.rectangle([8+jitter,8,w-8, h-8+jitter], outline=255, width=3)
        arr = np.array(img, dtype='float32')
        ruido = np.random.normal(0, 15, size).astype('float32')
        arr = np.clip(arr + ruido, 0, 255)
        X.append(arr.reshape(-1))
        y.append(1)
    return np.array(X), np.array(y)

# Intentar cargar desde carpeta; si falla, usar sintético
def cargar_o_sintetico(carpeta, size=(32,32)):
    carpeta = Path(carpeta)
    try:
        sub_ok = list((carpeta/'ok').glob('*.png'))
        sub_def = list((carpeta/'defecto').glob('*.png'))
        if len(sub_ok) == 0 or len(sub_def) == 0:
            raise FileNotFoundError
        X, y = [], []
        for etiqueta, sub in enumerate(['ok', 'defecto']):
            for ruta in (carpeta/sub).glob('*.png'):
                im = Image.open(ruta).convert('L').resize(size)
                X.append(np.array(im).astype('float32').reshape(-1))
                y.append(etiqueta)
        return np.array(X), np.array(y)
    except Exception:
        return generar_sintetico()

# Ejecutar ejemplo
Xproj, yproj = cargar_o_sintetico('/content/datos_piezas')  # en local, ajusta la ruta si procede
Xtr, Xte, ytr, yte = train_test_split(Xproj, yproj, test_size=0.25, random_state=0)

clf = LogisticRegression(max_iter=300)
clf.fit(Xtr, ytr)
yhat = clf.predict(Xte)
print('Accuracy proyecto (logística):', accuracy_score(yte, yhat))

In [ ]:
# Errores más comunes y visualización rápida
import matplotlib.pyplot as plt
idx_err = np.where(yhat != yte)[0]
print('Nº de errores:', len(idx_err))
if len(idx_err) > 0:
    k = min(8, len(idx_err))
    fig, axes = plt.subplots(1, k, figsize=(10,2))
    for ax, i in zip(axes, idx_err[:k]):
        ax.imshow(Xte[i].reshape(32,32))
        ax.set_title(f'{int(yte[i])}→{int(yhat[i])}')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

## 6) Conclusiones
- Los pasos esenciales están resueltos y comentados.
- Puedes sustituir el dataset sintético por imágenes reales en `ok/defecto`.
- Sugerencia: probar `MLPClassifier` en el proyecto y comparar resultados.

## Rúbrica de evaluación (proyecto)

| Criterio | Excelente (A) | Notable (B) | Aprobado (C) | Insuficiente (D) |
|---|---|---|---|---|
| **Comprensión del problema (20%)** | Problema bien definido, objetivos claros y medibles | Problema definido; objetivos parcialmente claros | Problema descrito de forma básica | Problema/confusión sin objetivos |
| **Metodología y proceso (30%)** | Pipeline reproducible, justificado y ordenado | Pipeline correcto con pequeñas lagunas | Pipeline funcional pero desordenado | Faltan pasos clave o no es reproducible |
| **Resultados y métricas (30%)** | Métricas adecuadas, análisis de errores y discusión | Métricas correctas, análisis limitado | Métricas básicas, sin análisis | Métricas inapropiadas o ausentes |
| **Comunicación y trabajo (20%)** | Informe claro, visualizaciones adecuadas, buen trabajo en equipo | Informe correcto, visualizaciones suficientes | Informe escueto, visualizaciones mínimas | Informe deficiente, sin colaboración |

**Entrega mínima**: código + README + requisitos + informe corto (2–4 páginas) con figuras y conclusiones.
